In [162]:
load(file = "pagerank.Rdata")
library("hash")

hash-2.2.6 provided by Decision Patterns



In [156]:
a = as.character(1)
pagerank[a]

1 
4.776286e-06

In [178]:
Movie_actor <- hash()

path = "movieid_actid.txt"
con = file(path, "r", encoding = "ISO-8859-1")

index = 1
while(TRUE){
    line = readLines(con, n = 1)
    p = unlist(strsplit(line, split = "\t"))
    if(length(line) == 0){
        break
    }
    
    key <- p[1]
    value <- p[2:length(p)]
    
    .set(Movie_actor, keys = key, values = value)
}

close(con)


Movie_rating <- hash()
path = "movieid_rate.txt"
con = file(path, "r", encoding = "ISO-8859-1")

index = 1

existing_movie <- c()
while(TRUE){
    line = readLines(con, n = 1)
    p = unlist(strsplit(line, split = "\t"))
    
    if(length(line) == 0){
        break
    }
    
    if(length(p) == 1){
        next
    }
    
    key <- p[1]
    
    existing_movie <- c(existing_movie, key)
    
    value <-as.numeric(p[2])
    
    .set(Movie_rating, keys = key, values = value)
}

close(con)

#index 1 larger than actual index

In [188]:

get('8', Movie_rating)
print(length(Movie_actor))
print(length(existing_movie))

[1] 7.5

[1] 203576
[1] 132977


In [277]:
linear_reg_judge <- function(subset_factor){
#    test_movie = c('12596', '48390', '100855')


    subset_movie <- sample(existing_movie, length(existing_movie)/subset_factor)


    train_data <- data.frame(matrix(ncol = 5, nrow = 0))
    colnames(train_data) <- c("rating", "pagerank_1","pagerank_2","pagerank_3","pagerank_4")

    for(i in subset_movie){

        if (!(i %in% test_movie)){

            actor_ids <- get(i, Movie_actor)

            pagerank_actor = c()

            for (id in actor_ids){
                pagerank_actor <- c(pagerank_actor, pagerank[id])
            }

            sort_pagerank = as.numeric(sort(pagerank_actor,decreasing=TRUE))

            c_feature_1 <- sort_pagerank[1]
            c_feature_2 <- sort_pagerank[2]
            c_feature_3 <- sort_pagerank[3]
            c_feature_4 <- sort_pagerank[4]

            c_rating <- get(i, Movie_rating)

            train_data[nrow(train_data)+1, ] <- c(c_rating, c_feature_1, c_feature_2, c_feature_3, c_feature_4)

        }

    }


    mod <- lm(rating ~ ., data = train_data)


    summary(mod)
    
    RSS <- c(crossprod(mod$residuals))
    MSE <- RSS / length(mod$residuals)
    RMSE <- sqrt(MSE)
    
    cat("RMSE", RMSE, '\n')
    for(i in test_movie){

        actor_ids <- get(i, Movie_actor)

        pagerank_actor = c()

        for (id in actor_ids){
            pagerank_actor <- c(pagerank_actor, pagerank[id])
        }

        c_feature <- sort(pagerank_actor,decreasing=TRUE)[1:4]

        features <- data.frame(matrix(ncol = 4, nrow = 1))
        colnames(features) <- c("pagerank_1","pagerank_2","pagerank_3","pagerank_4")

        features[1, ] <- as.numeric(c_feature)
        cat("predtict", predict(mod, features), '\n')

        
    }
}



In [278]:
test_movie = c('12596', '48390', '100855')

linear_reg_judge(1000)
cat('\n')
linear_reg_judge(500)
cat('\n')
linear_reg_judge(100)
cat('\n')
linear_reg_judge(50)
cat('\n')

    
   
        
 

        
    
    


RMSE 1.104741 
predtict 6.14015 
predtict 6.139553 
predtict 6.157364 

RMSE 1.264422 
predtict 6.013111 
predtict 6.047674 
predtict 6.118591 

RMSE 1.238669 
predtict 6.13774 
predtict 6.099596 
predtict 6.186541 

RMSE 1.226467 
predtict 6.107414 
predtict 6.125062 
predtict 6.152376 



In [251]:
for (i in test_movie){
    cat(get(i,Movie_actor), '\n')
}

446 999 1179 3202 3996 6292 6738 8400 9494 10638 10876 11494 12489 16235 19337 19588 21382 21383 23260 23770 26446 27513 28294 31155 32184 37806 38749 39949 40114 40214 43263 44277 45032 45085 45410 46141 46701 47320 48402 48711 54081 54827 56663 57833 58582 60548 68909 70523 74725 79319 82301 85846 86398 86803 87559 90083 91803 92569 93577 95634 95978 96563 99590 106456 110922 112202 
1938 2827 3036 3618 3839 3929 7502 14024 14421 14503 14593 17814 27158 27277 27493 27645 27903 28150 30023 30411 30905 32135 33226 34409 34671 38032 38782 39831 43458 43585 43817 44396 48728 52099 54802 55002 55958 56167 56972 58397 61170 63698 63915 71177 71208 74007 77622 78268 79159 81590 87740 88096 88814 89803 90742 94604 99226 100436 107739 108731 112827 112898 
4687 5380 12942 13523 14845 16587 20536 27125 27511 34036 41926 54750 56281 59075 79221 90693 95479 97174 98119 99656 105269 


In [274]:
for (i in test_movie){
    
    avg_ratings <- c()
    actors <- get(i,Movie_actor)
    
    for (actor in actors){
        curr_rating <- c()
        for (movie in existing_movie){
            tmp_actors <- get(movie, Movie_actor)
            if(actor %in% tmp_actors){
                tmp_rating <- get(movie, Movie_rating)
                if(is.null(tmp_rating) || is.na(tmp_rating)){
                    cat(movie)
                }
                curr_rating <- c(curr_rating, tmp_rating)
            }
        }
        
        if(length(curr_rating) != 0){
            avg_ratings <- c(avg_ratings, mean(curr_rating))
        }
    }
    
    cat("predict", mean(avg_ratings), '\n')
}

predict 6.362715 
predict 6.424666 
predict 6.856822 


In [264]:
a = c(1,2,3,NA)
b <-na.omit(a)

In [265]:
b

[1] 1 2 3
attr(,"na.action")
[1] 4
attr(,"class")
[1] "omit"